In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [112]:
!git remote add origin https://github.com/rishikasrinivas/CSE144Group11.git
!git branch -M main
!git push -u origin main

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


Loading Labels Into List

In [ ]:
def load_gloss_words(filepath):
  bag_of_words = []
  with open(filepath) as f:
    for line in f.readlines():
      tab_ind = line.find('\t')
      new_line_ind = line.find('\n')

      word = line[tab_ind + 1: new_line_ind]
      #print(word)
      bag_of_words.append(word)

  return bag_of_words

In [ ]:
class_list_file_path = "/content/drive/MyDrive/CSE 144 Study Group/Data/wlasl_class_list.txt"
gloss_words = load_gloss_words(class_list_file_path)

Loading Missing Video ID's (some of the videos in the json are missing)

In [ ]:
def load_missing_videos_ids(filepath):
  missing_vid_ids = []
  with open(filepath) as f:
    for line in f.readlines():
      missing_vid_ids.append(int(line[:-1]))

  return missing_vid_ids

In [ ]:
missing_video_ids= "/content/drive/MyDrive/CSE 144 Study Group/Data/missing.txt"
missing_vid_ids= load_missing_videos_ids(missing_video_ids)

In [ ]:
missing_vid_ids

Loading In JSON

In [ ]:
import json
json_file_path = "/content/drive/MyDrive/CSE 144 Study Group/Data/WLASL_v0.3.json"
f = open(json_file_path)
data = json.load(f)

In [ ]:
def load_json_to_dict(data):
  gloss_word_to_inst_dict = {}
  for gloss_words_info in data:
    vid_ids = []
    for instance_struct in gloss_words_info['instances']:
      vid_id = instance_struct['video_id']
      vid_ids.append(int(vid_id))
    if gloss_words_info['gloss'] not in gloss_word_to_inst_dict.keys():
      gloss_word_to_inst_dict[gloss_words_info['gloss']] = vid_ids
  return gloss_word_to_inst_dict

In [ ]:
def remove_missing_video_ids(instances_dict,missing_vid_ids):
  for gloss_word in instances_dict.keys():
    vid_ids = instances_dict[gloss_word]
    for vid_id in vid_ids:
      if vid_id in missing_vid_ids:
        instances_dict[gloss_word].remove(vid_id)
  return instances_dict

In [ ]:
gloss_word_and_inst_dict = load_json_to_dict(data)
new_id_dict = remove_missing_video_ids(gloss_word_and_inst_dict, missing_vid_ids)

def identify_differences_in_altered_vs_orig(gloss_word_and_inst_dict_orig, new_id_dict):
  orig_num_vids=0
  for gloss in gloss_word_and_inst_dict_orig.keys():
    orig_num_vids += len(gloss_word_to_inst_dict[gloss])

  after_remove_missing = 0

  for gloss in new_id_dict.keys():
    after_remove_missing += len(new_id_dict[gloss])

  return orig_num_vids, after_remove_missing,  after_remove_missing - orig_num_vids
orig_num_vids,after_remove_missing, difference = identify_differences_in_altered_vs_orig(gloss_word_and_inst_dict, new_id_dict)

print(orig_num_vids,after_remove_missing, difference)


{'book': [69241, 68011, 68012, 70266, 7086, 7069, 7089, 7091, 7093, 7068, 7095, 7097, 7070, 7099, 7072, 67424, 7074, 7076, 7078, 7080, 7082, 7084], 'drink': [69302, 65539, 68538, 68660, 17725, 17727, 17710, 17730, 17732, 17733, 65540, 17734, 17711, 17712, 17713, 17715, 17717, 17709, 17719, 17720, 17721, 17722, 17723, 17724], 'computer': [68028, 12328, 12330, 12312, 12332, 12335, 12311, 12338, 12313, 12314, 12315, 12316, 12317, 12318, 12319, 12320, 12321, 12323, 12326, 12327], 'before': [70348, 5744, 5728, 5748, 5749, 5750, 5729, 5730, 65167, 5731, 5732, 5733, 5734, 5736, 5727, 5739, 5740, 5741, 5742, 5743], 'chair': [70230, 70263, 9865, 9848, 9869, 9849, 9850, 9851, 65328, 9854, 9856, 9857, 9859, 9861, 9863], 'go': [69345, 24955, 24956, 24941, 24960, 24961, 24962, 24969, 24971, 65824, 24973, 24943, 24946, 24947, 24940, 24950, 24952, 24954], 'clothes': [68870, 11327, 11310, 11330, 11311, 11313, 11315, 11309, 11317, 11319, 11321, 11323, 11325], 'who': [69534, 68183, 63240, 63242, 63226, 

Model to use

In [ ]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Predicted class: Egyptian cat
